In [1]:
pwd

'/home/greenman225/Code/github/donkey_golf/jupyter_notebooks'

In [2]:
cd ..

/home/greenman225/Code/github/donkey_golf


In [3]:
ls

donkey_golf/  jupyter_notebooks/  README.md  requirements.txt  run.py


In [4]:
cd donkey_golf/

/home/greenman225/Code/github/donkey_golf/donkey_golf


In [5]:
ls

all_things_data/  forms.py     __pycache__/  static/
config.py         __init__.py  routes.py     templates/
data_utils.py     models.py    site.db       yaml_files/


In [6]:
cd all_things_data/

/home/greenman225/Code/github/donkey_golf/donkey_golf/all_things_data


In [7]:
import sql_classes

ModuleNotFoundError: No module named 'donkey_golf'